# <font color=orange><div align="center">SDP Project - Question 3</div></font>

### <font color=orange><div align="center">15/12/2025</div></font>
### <font color=orange><div align="center">Ouissal BOUTOUATOU - Alae TAOUDI - Mohammed SBAIHI</div></font>


Refer to [Problem Formulation](Question3.md) for rigorous mathematical formulation of the problem

In [47]:
from gurobipy import *

## Instanciation

In [48]:
CANDIDATES = {
    "x": [85, 81, 71, 69, 75, 81, 88],
    "y": [81, 81, 75, 63, 67, 88, 95],
    "z": [74, 89, 74, 81, 68, 84, 79],
    "t": [74, 71, 84, 91, 77, 76, 73],
    "u": [72, 75, 66, 85, 88, 66, 93],
    "v": [71, 73, 63, 92, 76, 79, 93],
    "w": [79, 69, 78, 76, 67, 84, 79],
    "w'": [57, 76, 81, 76, 82, 86, 77]
}

WEIGHTS = [8, 7, 7, 6, 6, 5, 6]
COURSES = ["A", "B", "C", "D", "E", "F", "G"]

In [49]:
def pros_and_cons(x,y):
    """Returns the pros(x,y) and cons(x, y) mappings: Course -> Contribution to x > y
    Inputs: x, y students
    N.B: courses are annotated using integers instead of alphabets
    """
    assert x in CANDIDATES, f"{x} isn't in candidates"
    assert y in CANDIDATES, f"{y} isn't in candidates"

    pros = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) > 0
    }

    cons = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) < 0
    }
    return pros, cons

## Part 1: Demonstrating that no (1-m) explanation exists for u > v

In [50]:
# Analysis for u > v
PROS_UV, CONS_UV = pros_and_cons("u", "v")
print("Pros(u, v):", PROS_UV)
print("Cons(u, v):", CONS_UV)
print("\nDetailed contributions:")
for course_idx, contrib in PROS_UV.items():
    print(f"  {COURSES[course_idx]}: +{contrib}")
for course_idx, contrib in CONS_UV.items():
    print(f"  {COURSES[course_idx]}: {contrib}")

Pros(u, v): {0: 8, 1: 14, 2: 21, 4: 72}
Cons(u, v): {3: -42, 5: -65}

Detailed contributions:
  A: +8
  B: +14
  C: +21
  E: +72
  D: -42
  F: -65


### Exhaustive verification: Check if any (1-m) trade-offs are valid

For a (1-m) explanation to exist, we need to cover all cons courses using valid trade-offs where each pros course is paired with a subset of cons courses such that their sum is positive.

In [51]:
from itertools import combinations

print("Testing all possible (1-m) trade-offs:\n")
print("=" * 80)

valid_tradeoffs = {}

# For each pros course, check all possible subsets of cons
for p in PROS_UV:
    pros_course = COURSES[p]
    pros_contrib = PROS_UV[p]
    
    print(f"\nPros course {pros_course} (+{pros_contrib}):")
    
    valid_for_this_pros = []
    
    # Check all non-empty subsets of cons
    for r in range(1, len(CONS_UV) + 1):
        for cons_subset in combinations(CONS_UV.keys(), r):
            cons_contrib = sum(CONS_UV[c] for c in cons_subset)
            total = pros_contrib + cons_contrib
            
            cons_names = [COURSES[c] for c in cons_subset]
            
            if total > 0:
                cons_list = ', '.join(cons_names)
                print(f"  [OK] With {{{cons_list}}}: {pros_contrib} + ({cons_contrib}) = {total} > 0")
                valid_for_this_pros.append(cons_subset)
            else:
                cons_list = ', '.join(cons_names)
                print(f"  [X] With {{{cons_list}}}: {pros_contrib} + ({cons_contrib}) = {total} ≤ 0")
    
    valid_tradeoffs[p] = valid_for_this_pros

print("\n" + "=" * 80)
print("SUMMARY OF VALID (1-m) TRADE-OFFS:")
print("=" * 80)

for p, tradeoffs in valid_tradeoffs.items():
    pros_course = COURSES[p]
    if tradeoffs:
        print(f"\nPros {pros_course} can form {len(tradeoffs)} valid trade-off(s):")
        for cons_subset in tradeoffs:
            cons_names = [COURSES[c] for c in cons_subset]
            cons_list = ', '.join(cons_names)
            print(f"  - ({pros_course}, {{{cons_list}}})")
    else:
        print(f"\nPros {pros_course}: NO valid trade-offs")

Testing all possible (1-m) trade-offs:


Pros course A (+8):
  [X] With {D}: 8 + (-42) = -34 ≤ 0
  [X] With {F}: 8 + (-65) = -57 ≤ 0
  [X] With {D, F}: 8 + (-107) = -99 ≤ 0

Pros course B (+14):
  [X] With {D}: 14 + (-42) = -28 ≤ 0
  [X] With {F}: 14 + (-65) = -51 ≤ 0
  [X] With {D, F}: 14 + (-107) = -93 ≤ 0

Pros course C (+21):
  [X] With {D}: 21 + (-42) = -21 ≤ 0
  [X] With {F}: 21 + (-65) = -44 ≤ 0
  [X] With {D, F}: 21 + (-107) = -86 ≤ 0

Pros course E (+72):
  [OK] With {D}: 72 + (-42) = 30 > 0
  [OK] With {F}: 72 + (-65) = 7 > 0
  [X] With {D, F}: 72 + (-107) = -35 ≤ 0

SUMMARY OF VALID (1-m) TRADE-OFFS:

Pros A: NO valid trade-offs

Pros B: NO valid trade-offs

Pros C: NO valid trade-offs

Pros E can form 2 valid trade-off(s):
  - (E, {D})
  - (E, {F})


### Analysis: Can we cover all cons courses?

In [52]:
print("\n" + "=" * 80)
print("COVERAGE ANALYSIS:")
print("=" * 80)

# Check if we can cover all cons
all_cons = set(CONS_UV.keys())
print(f"\nCons courses to cover: {{{', '.join([COURSES[c] for c in all_cons])}}}")

# For each pros, show what cons it can validly cover
print("\nCoverage possibilities:")
for p, tradeoffs in valid_tradeoffs.items():
    if tradeoffs:
        # Get all cons that can be covered by this pros
        coverable_cons = set()
        for cons_subset in tradeoffs:
            coverable_cons.update(cons_subset)
        
        pros_course = COURSES[p]
        cons_names = [COURSES[c] for c in coverable_cons]
        print(f"  {pros_course} can cover: {{{', '.join(cons_names)}}}")

# Critical observation
print("\n" + "=" * 80)
print("CRITICAL OBSERVATION:")
print("=" * 80)

# Check if E can cover both D and F simultaneously
if 4 in valid_tradeoffs:  # E is index 4
    can_cover_both = any(
        set(cons_subset) == all_cons 
        for cons_subset in valid_tradeoffs[4]
    )
    
    print(f"\nCourse E can cover D alone: OK")
    print(f"Course E can cover F alone: OK")
    print(f"Course E can cover both D and F simultaneously: {can_cover_both}")
    
    if not can_cover_both:
        print("\n[!] Course E (the only pros with valid trade-offs) cannot cover")
        print("    both cons courses D and F at the same time with a positive sum.")
        print("    Since other pros courses have NO valid trade-offs, it's impossible")
        print("    to construct a (1-m) explanation that covers all cons courses.")
        print("\n[CONCLUSION] NO (1-m) explanation exists for u > v")


COVERAGE ANALYSIS:

Cons courses to cover: {D, F}

Coverage possibilities:
  E can cover: {D, F}

CRITICAL OBSERVATION:

Course E can cover D alone: OK
Course E can cover F alone: OK
Course E can cover both D and F simultaneously: False

[!] Course E (the only pros with valid trade-offs) cannot cover
    both cons courses D and F at the same time with a positive sum.
    Since other pros courses have NO valid trade-offs, it's impossible
    to construct a (1-m) explanation that covers all cons courses.

[CONCLUSION] NO (1-m) explanation exists for u > v


## Part 2: Linear Program for (m-1) Explanations

### Definition:
A (m-1) trade-off is a pair ({P₁, ..., Pₘ}, C) where multiple pros courses are paired with one cons course such that their sum is positive.

We will now formulate and solve a linear program to find (m-1) explanations.

### Finding a (m-1) explanation for y > z

In [53]:
PROS, CONS = pros_and_cons("u", "v")
print("The Pros Set:", PROS)
print("The Cons Set:", CONS)

The Pros Set: {0: 8, 1: 14, 2: 21, 4: 72}
The Cons Set: {3: -42, 5: -65}


## Variables

Let $P = |\texttt{pros}(x, y)|$ and $C = |\texttt{cons}(x, y)|$. <br>

We define $X \in \{0, 1\}^{P \times C}$ such that:

$$ x_{p, c} = \begin{cases} 1 & \text{if } P_p \text{ is associated with } C_c \text{ in the (m-1) explanation} \\ 0 & \text{otherwise} \end{cases} $$

**Note**: For (m-1) explanations:
- **ALL cons must be covered** (each cons must have at least one pros)
- Multiple pros can be grouped to cover one cons
- Not all pros need to be used

In [54]:
m = Model("Problem_3")

# x[p,c] variables: association between pros p and cons c
VarDictX = {(p, c) : m.addVar(vtype = GRB.BINARY, name=f'x_{p}_{c}')
                    for p in PROS
                    for c in CONS}

display(VarDictX)

{(0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (4, 3): <gurobi.Var *Awaiting Model Update*>,
 (4, 5): <gurobi.Var *Awaiting Model Update*>}

## Constraints

1. Each pro course can be associated to **at most** one cons course (optional participation)

    $$ \forall p \in [1, P], \sum_{c=1}^{C} x_{p, c} \leq 1 $$

In [55]:
OPTIONALCONSTDIC = {
    p: m.addConstr(
        quicksum([VarDictX[(p,c)] for c in CONS]) <= 1
    )
    for p in PROS
}

2. Each cons course **must have at least one pros** associated with it

    $$ \forall c \in [1, C], \sum_{p=1}^{P} x_{p, c} \geq 1 $$

In [56]:
COVERAGECONSTDIC = {
    c: m.addConstr(
        quicksum([VarDictX[(p, c)] for p in PROS]) >= 1
    )
    for c in CONS
}

3. Trade-off validity: For each cons course, the sum of all associated pros contributions and the cons contribution must be strictly positive

    $$ \forall c \in [1, C], \sum_{p=1}^{P} x_{p, c} \cdot p_p + c_c \geq \epsilon $$
    
    where $\epsilon > 0$ ensures strict positivity

In [57]:
EPSILON = 0.01

VALIDITYCONSTDIC = {
    c: m.addConstr(
        quicksum([VarDictX[(p, c)] * PROS[p] for p in PROS]) + CONS[c] >= EPSILON
    )
    for c in CONS
}

## Objective Function

Minimize the total number of pros courses used in the explanation:

$$ \min \sum_{p=1}^{P} \sum_{c=1}^{C} x_{p,c} $$

In [58]:
m.setObjective(quicksum([VarDictX[(p,c)] for p in PROS for c in CONS]), GRB.MINIMIZE)

## Solve the Model

In [59]:
m.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11+.0 (26200.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1360P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Academic license 2754426 - for non-commercial use only - registered to ou___@student-cs.fr
Optimize a model with 8 rows, 8 columns and 24 nonzeros (Min)
Model fingerprint: 0x0d88c456
Model has 8 linear objective coefficients
Variable types: 0 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+0

## Results and Interpretation

In [60]:
if m.status == GRB.OPTIMAL:
    print("=" * 80)
    print("OPTIMAL SOLUTION FOUND!")
    print("=" * 80)
    print(f"\nMinimum number of pros used: {int(m.objVal)}")
    print("\n" + "=" * 80)
    print("EXPLANATION (m-1):")
    print("=" * 80)
    
    # Extract the solution - group by cons
    for c in CONS:
        # Find all pros courses associated with this cons
        associated_pros = [p for p in PROS if VarDictX[(p, c)].X == 1]
        
        if associated_pros:  # If this cons has pros associated
            # Calculate total contribution
            total_contrib = sum(PROS[p] for p in associated_pros) + CONS[c]
            
            # Display the trade-off
            cons_course = COURSES[c]
            pros_courses = [COURSES[p] for p in associated_pros]
            
            print(f"\nTrade-off: ({{{', '.join(pros_courses)}}}, {cons_course})")
            for p in associated_pros:
                print(f"  - Pros contribution [{COURSES[p]}]: {PROS[p]:+d}")
            print(f"  - Cons contribution [{COURSES[c]}]: {CONS[c]:+d}")
            print(f"  - Total contribution: {total_contrib:+d}")
            print(f"  - Valid: {total_contrib > 0}")
    
    print("\n" + "=" * 80)
    print("VERIFICATION:")
    print("=" * 80)
    
    # Verify all cons are covered
    covered_cons = set()
    used_pros = set()
    for c in CONS:
        for p in PROS:
            if VarDictX[(p, c)].X > 0.5:
                covered_cons.add(c)
                used_pros.add(p)
    
    cons_names = [COURSES[c] for c in CONS]
    covered_cons_names = [COURSES[c] for c in covered_cons]
    used_pros_names = [COURSES[p] for p in used_pros]
    
    print(f"Cons courses to cover: {{{', '.join(cons_names)}}}")
    print(f"Cons courses covered: {{{', '.join(covered_cons_names)}}}")
    print(f"All cons covered: {covered_cons == set(CONS.keys())}")
    print(f"\nPros courses used: {{{', '.join(used_pros_names)}}}")
    print(f"Total pros used: {len(used_pros)}/{len(PROS)}")
    
elif m.status == GRB.INFEASIBLE:
    print("=" * 80)
    print("MODEL IS INFEASIBLE!")
    print("=" * 80)
    print("\nNo (m-1) explanation exists for this comparison.")
    print("This means it's impossible to construct valid trade-offs that cover all cons courses.")
    
else:
    print(f"Optimization ended with status: {m.status}")

OPTIMAL SOLUTION FOUND!

Minimum number of pros used: 4

EXPLANATION (m-1):

Trade-off: ({A, B, C}, D)
  - Pros contribution [A]: +8
  - Pros contribution [B]: +14
  - Pros contribution [C]: +21
  - Cons contribution [D]: -42
  - Total contribution: +1
  - Valid: True

Trade-off: ({E}, F)
  - Pros contribution [E]: +72
  - Cons contribution [F]: -65
  - Total contribution: +7
  - Valid: True

VERIFICATION:
Cons courses to cover: {D, F}
Cons courses covered: {D, F}
All cons covered: True

Pros courses used: {A, B, C, E}
Total pros used: 4/4
